# Custom populations

Custom populations can be created either by piecing together existing populations (spatial and luminosity populations) or building them from scratch with distributions.

**popsynth** comes loaded with many combinations of typical population distributions. However, we demonstrate here how to create your own.

## Creating distributions

The population samplers rely on distributions. Each population has an internal spatial and luminosity distribution. For example, lets look at a simple spatial distribution:


In [2]:
%matplotlib notebook


import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(context='notebook', fscale=1, grid=False)
green = "#1DEBA6"
red = "#FF0059"
yellow = "#F6EF5B"


import popsynth

import warnings
warnings.simplefilter('ignore')

In [3]:
from popsynth.distribution import SpatialDistribution


class SphericalDistribution(SpatialDistribution):
    def __init__(self, r_max=10, seed=1234, name="sphere", form=None, truth={}):

        
         # the latex formula for the ditribution
        form = r"4 \pi r2"
        super(SphericalDistribution, self).__init__(
            r_max=r_max, seed=seed, name=name, form=form, truth=truth
        )

    def differential_volume(self, r):

        return 4 * np.pi * r * r

    def transform(self, L, r):

        return L / (4.0 * np.pi * r * r)


We simply define the differential volume and how luminosity is transformed to flux in the metric. Here, we have a simple sphere out to some *r_max*. We can of course subclass this object and add a normalization.


Now we define a luminosity distribution.

In [4]:
from popsynth.distribution import LuminosityDistribution


class ParetoDistribution(LuminosityDistribution):
    def __init__(self, Lmin, alpha, seed=1234, name="pareto"):

        # the truth parameters for storage
        truth = dict(Lmin=Lmin, alpha=alpha)

        # the latex formula for the ditribution
        lf_form = r"\frac{\alpha L_{\rm min}^{\alpha}}{L^{\alpha+1}}"
        
        
        super(ParetoDistribution, self).__init__(
            seed=seed, name=name, form=lf_form, truth=truth
        )

        # since we have extra parameters, we need to tell the class about them
        self._construct_distribution_params(Lmin=Lmin, alpha=alpha)

    def phi(self, L):
        
        # the actual function, only for plotting 

        out = np.zeros_like(L)

        idx = L >= self.Lmin

        out[idx] = self.alpha * self.Lmin ** self.alpha / L[idx] ** (self.alpha + 1)

        return out

    def draw_luminosity(self, size=1):
        # how to sample the latent parameters
        return (np.random.pareto(self._params["alpha"], size) + 1) * self._params[
            "Lmin"
        ]